<a href="https://colab.research.google.com/github/mohanvamsitrade/Gen_AI_Projects/blob/main/Restaurant_Advisor_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from secret_key import huggingface_key1

import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = huggingface_key1

In [ ]:
!pip install langchain

In [ ]:
!pip install tiktoken

In [ ]:
!pip install cohere


In [5]:
pip install --upgrade typing-extensions==4.7

In [ ]:
!pip install --upgrade llmx

In [7]:
!pip install huggingface

In [9]:
!pip -q install langchain huggingface_hub transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.2 MB/s eta 0:00:00


In [42]:
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id = "google/flan-t5-xxl", model_kwargs= {'temperature':0.6})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [49]:
name = llm("I want to open a restaurant for Indian food. Suggest a fency name for this")
print(name)

menu = llm("Give me a 5 menu names for Indian restaurant")
print(menu)

Indian Fusion
tandoori chicken tikka nan sag aloo dhaniya korma saag paneer


In [67]:
from langchain.prompts import PromptTemplate

prompt_template_name  = PromptTemplate(
    input_variables=['cusine'], template = "I want to open a restaurant for {cusine} food. Suggest a fency name for this"
)

prompt_template_name.format(cusine = 'English')

prompt_template_menu = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest 6 menu items for {restaurant_name}. Return it as a comma separated list"
)

In [68]:
from langchain.chains import LLMChain

name_chain = LLMChain(llm = llm, prompt= prompt_template_name)
menu_chain = LLMChain(llm = llm, prompt= prompt_template_menu)
name_chain.invoke('Indian')
menu_chain.invoke('Indian')

{'restaurant_name': 'Indian', 'text': ',,,, '}

In [69]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain, menu_chain])

response = chain.invoke("Mexican")
print(response)



{'input': 'Mexican', 'output': 'fajitas, enchiladas, burrito, carnitas, chile relleno'}


In [77]:
prompt_template_name  = PromptTemplate(
    input_variables=['cusine'],
    template = "I want to open a restaurant for {cusine} food. Suggest a fency name for this"
)

prompt_template_menu = PromptTemplate(
    input_variables=['restaurant_name'],
    template="Suggest 6 menu items for {restaurant_name}. Return it as a comma separated list"
)

name_chain = LLMChain(llm = llm, prompt= prompt_template_name,output_key="restaurant_name")
menu_chain = LLMChain(llm = llm, prompt= prompt_template_menu, output_key= "menu_items")

In [88]:
from langchain.chains import SequentialChain

new_chain = SequentialChain(
    chains = [name_chain, menu_chain],
    input_variables=['cusine'],
    output_variables= ['restaurant_name','menu_items']
)

new_chain({'cusine':'Mexican'})

{'cusine': 'Mexican',
 'restaurant_name': 'Mexican food restaurant',
 'menu_items': 'fajitas, enchiladas, burrito, carnitas, chile relleno'}